In [2]:
from pandas import DataFrame as data_frame
from bcolz import open as bcolz_open
from json import load as json_load
from bs4 import BeautifulSoup
from numpy import array
from pickle import load
from re import sub
import nltk.data

glove_path = 'C:\\Users\\Karthik\\Desktop\\sentiment_analysis\\imdb\\glove\\'

In [3]:
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras._impl.keras.optimizers import Adam
from tensorflow.python.keras._impl.keras.preprocessing import sequence

In [4]:
#Load the vectors from GloVe
vecs = bcolz_open(glove_path+ 'glove.6B.50d.dat')[:]
words = load(open(glove_path+'glove.6B.50d_words.pkl','rb'))
wordidx = load(open(glove_path+'glove.6B.50d_idx.pkl','rb'))

In [5]:
#User Defined function to retrieve Word Vector
def w2v(w): return vecs[wordidx[w]]

In [6]:
def review_to_wordlist(review):
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = sub("[^a-zA-Z\-]"," ", review_text)
    words = review_text.lower().split()
    words += '.'
    return(words)

In [7]:
#punkt tokenizer for sentence splitting
#nltk.download()   
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences( review, tokenizer):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence))
    return sentences

In [7]:
with open(r'C:\Users\Karthik\Desktop\sentiment_analysis\imdb\idx.json') as f:
    idx = json_load(f)

In [8]:
idx2word = {v: k for k, v in idx.items()}

In [9]:
seq_len = 1500

In [10]:
vocab_size = 15000

In [11]:
#Should EoS-'period' be handled here or within the rev2sentnc func?
def reformat_dataset_list_of_words(dataset):
    X_revw_indx = []
    X_reviews = []
    for record in dataset:
        review=[]
        indices = []
        sentences = []
        sentences += review_to_sentences(record, tokenizer)
        for sentence in sentences:
            for word in sentence:
                review.append(word)
                indices.append(idx[word])
        X_reviews.append(review)
        X_revw_indx.append(indices)
    return X_reviews, X_revw_indx


In [12]:
model_path = "C:\\Users\\Karthik\\Desktop\\sentiment_analysis\\imdb\\"

In [13]:
model = load_model(
    filepath = model_path+'cnn_model_glove50.h5',
    custom_objects=None,
    compile=True
)

In [14]:
model.load_weights(
    filepath = model_path+"kg_glove50_1.h5",
    by_name=False
)

In [15]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [16]:
input_text = "One stock that might be an intriguing choice for investors right now is Marathon Oil Corporation. This is because this security in the Oil and Gas - integrated - United States space is seeing bad earnings estimate revision activity, and is from Industry Rank perspective."


In [17]:
X=data_frame({'review':[input_text]})
_, X_revw_indx = reformat_dataset_list_of_words(X['review'])
X_test = [array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in X_revw_indx]
X_test = sequence.pad_sequences(X_test, maxlen=seq_len, value=0)
pred = model.predict(
    X_test,
    batch_size=None,
    verbose=0
)

In [ ]:
pred

array([[ 0.38903007]], dtype=float32)

In [ ]:
while(True):
    input_text = input("Please enter a review:\n")
    if input_text == "q!":
        break
    X=data_frame({'review':[input_text]})
    _, X_revw_indx = reformat_dataset_list_of_words(X['review'])
    X_test = [array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in X_revw_indx]
    X_test = sequence.pad_sequences(X_test, maxlen=seq_len, value=0)
    pred = model.predict(
        X_test,
        batch_size=None,
        verbose=0
    )
    print("Sentiment score:\n", pred)
    

Please enter a review:
dumb
Sentiment score:
 [[ 0.23291089]]
Please enter a review:
foul
Sentiment score:
 [[ 0.25694355]]
Please enter a review:
profit
Sentiment score:
 [[ 0.47700569]]
Please enter a review:
loss
Sentiment score:
 [[ 0.2344418]]
